In [1]:
import pandas as pd
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

df = pd.read_csv("/content/sample_data/habr_articles.csv")
df =  df.dropna(subset=["Текст"])
df.info()
text = df['Текст']
input_sequences = []

def clean(text):
    text = re.sub(r'[!-,.«»:?;]', '', text.lower())
    text = re.sub(r'\s+', ' ',text)
    return text

text = text.apply(clean)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1


for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences,
                                maxlen=max_sequence_length,
                                padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


<class 'pandas.core.frame.DataFrame'>
Index: 989 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Название        989 non-null    object
 1   Время чтения    989 non-null    object
 2   Просмотры       989 non-null    object
 3   Текст           989 non-null    object
 4   Ключевые слова  989 non-null    object
 5   Ссылка          989 non-null    object
dtypes: object(6)
memory usage: 54.1+ KB


# Нейросетки


In [ ]:
model_rnn = Sequential()
model_rnn.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_rnn.add(SimpleRNN(100))
model_rnn.add(Dense(total_words, activation='softmax'))
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_rnn.fit(X, y, epochs=10, verbose=0)

model_gru = Sequential()
model_gru.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_gru.add(GRU(100))
model_gru.add(Dense(total_words, activation='softmax'))
model_gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_gru.fit(X, y, epochs=10, verbose=0)

model_lstm = Sequential()
model_lstm.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(total_words, activation='softmax'))
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_lstm.fit(X, y, epochs=10, verbose=0)

def generate_text(seed_text, model, max_sequence_len, num_words):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = pd.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

generated_text_rnn = generate_text("Это", model_rnn, max_sequence_length, num_words=10)
generated_text_gru = generate_text("Это", model_gru, max_sequence_length, num_words=10)
generated_text_lstm = generate_text("Это", model_lstm, max_sequence_length, num_words=10)

print("Сгенерированный текст (SimpleRNN):", generated_text_rnn)
print("Сгенерированный текст (GRU):", generated_text_gru)
print("Сгенерированный текст (LSTM):", generated_text_lstm)

Статистическая модель марковская цепь

In [2]:
import pandas as pd
import re
import random
from collections import defaultdict

df = pd.read_csv("/content/sample_data/habr_articles.csv")
df = df.dropna(subset=["Текст"])
df = df.drop(columns=['Название', 'Время чтения', 'Просмотры', 'Ключевые слова', 'Ссылка'])
text = df['Текст']

def clean(text):
    text = re.sub(r'[!-,.«»:?;—]', '', text.lower())
    text = re.sub(r'\s+', ' ', text)
    return text

texts = text.apply(clean)

def build_markov_chain(texts, n=2):
    chain = defaultdict(list)
    for text in texts:
        tokens = text.split()
        for i in range(len(tokens) - n + 1):
            key = tuple(tokens[i:i + n - 1])
            next_word = tokens[i + n - 1]
            chain[key].append(next_word)
    return chain

def generate_text_markov(chain, n=2, start_text='', max_words=50):
    if start_text:
        current_state = tuple(start_text.split()[:n-1])
    else:
        current_state = random.choice(list(chain.keys()))

    generated_text = list(current_state)

    for _ in range(max_words):
        next_words = chain.get(current_state, [])
        if not next_words:
            break
        next_word = random.choice(next_words)
        generated_text.append(next_word)
        current_state = tuple(generated_text[-(n-1):])

    return ' '.join(generated_text)

def generate_multiple_texts(chain, n=2, max_words=50, number_offers=10):
    texts = []
    for _ in range(number_offers):
        generated_text = generate_text_markov(chain, n=n, start_text='', max_words=max_words)
        texts.append(generated_text)
    return texts


n = 3
markov_chain = build_markov_chain(texts, n)
generated_texts = generate_multiple_texts(markov_chain, n=n, max_words=50, number_offers=10)

for i, t in enumerate(generated_texts, 1):
    print(f"Текст {i}:\n{t}\n")


Текст 1:
данных используемых для повышения безопасности ит-систем северной кореи велкам

Текст 2:
по 4 уровню доверия

Текст 3:
вроде ci/cd точечное внедрение инструментов и фреймворков исходя из меняющегося контекста по заранее определенным правилам особенность именно безлимитной версии покера в том что её проектируют в первую очередь опирается на личный опыт вхождения в it и представить не мог представить что эта тема привлекает внимание исследователей в области автоматизации объясню просто как

Текст 4:
к решению задач кибербезопасности приводят к созданию бота

Текст 5:
систем 1с в качестве самого сервиса будем использовать nextjs shadcn для интерфейса и sms api с подтверждением договоренностей и следующими шагами

Текст 6:
мер мотивации недостаток практики и скучная слишком сухо преподнесенная теория – типичные примеры может показаться сложной задачей но если запустить их ещё в dosbox у себя на машине можно - но покажем ассемблеры для нескольких популярных архитектур микроконтролл